In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorboardX import SummaryWriter
from IPython.display import clear_output


/home/henrytsui/anaconda3/envs/IDS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_PATH = 'bert-base-uncased'
MAX_LEN = 64
DEVICE = "cuda"
BZ = 128
WD = 1e-9
LR = 1e-5
EPS = 50
TMAX = 15
COMMENT = f"lr{LR}-B{BZ}-EPS{EPS}-TOU{TMAX}"

In [15]:
merge = pd.read_csv("../data/new_pred.csv")

In [27]:
class Stock(Dataset):
    def __init__(self, df) -> None:
        self.df = df
        
        self.day = self.df["datentime"]
        self.day = set(self.day)
        self.price = pd.concat([self.df["preprice"], self.df["nowprice"]], axis=1)
        self.price = self.price.to_numpy()
        
        self.feature = pd.concat([self.df[f"f{idx:02d}"] for idx in range(1, 17)], axis=1)
        for com
        # self.feature = self.feature.to_numpy()
        
        print(self.price.shape)
        
        

    def __getitem__(self, idx):
        pass
        # return self.price[idx], self.feature[idx], self.

    def __len__(self):
        return self.df.shape[0]
dataset = Stock(merge)

(25278, 2)


In [29]:
s = next(iter(dataset))
# print(s.shape)
print(s)

(array([90.79, 96.72]), array([-1.91845191, 16.25706863, 15.89408398, -1.17809319, -1.68019831,
        0.21510312, -0.84030849, 20.51179123, -0.42434102, -0.03772284,
       -0.1581393 , -0.29264325, -4.73116779, -0.09351555,  0.12904562,
       -0.05970213]))
